This notebook contains the data prep for a Tableau dashboard that explores how immigration enforcement changed within each presidential administration from 1925 - 2016. 

The immigration data was acquired from the Department of Homeland security
Presidential term data was acquired from Kaggle
Base population data was acquired from Wikipedia

In [ ]:
import pandas as pd
import itertools
import numpy as np
import datetime as dt

#Data for Presidential term and party information
terms = pd.read_csv('/Users/jackhulbert/Desktop/Data Science Projects/Immigration/Data/us_presidents.csv')

#Data for immigration enforcement actions, sourced from DHS.

immigration = pd.read_csv('/Users/jackhulbert/Desktop/Data Science Projects/Immigration/Data/ImmigrationEnforcement.csv')

In [71]:
immigration.head()

,Year,Enforcement Status,Records
0,1925,Apprehended,22199.0
1,1926,Apprehended,12735.0
2,1927,Apprehended,16393.0
3,1928,Apprehended,23566.0
4,1929,Apprehended,32711.0


In [72]:
terms.head()

,Unnamed: 0,S.No.,start,end,president,prior,party,vice,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,1,"April 30, 1789","March 4, 1797",George Washington,Commander-in-Chief of the Continental Army ...,Nonpartisan [13],John Adams,NaN,NaN,NaN,NaN,NaN
1,1,2,"March 4, 1797","March 4, 1801",John Adams,1st Vice President of the United States,Federalist,Thomas Jefferson,NaN,NaN,NaN,NaN,NaN
2,2,3,"March 4, 1801","March 4, 1809",Thomas Jefferson,2nd Vice President of the United States,Democratic- Republican,Aaron Burr,NaN,NaN,NaN,NaN,NaN
3,3,4,"March 4, 1809","March 4, 1817",James Madison,5th United States Secretary of State (1801–...,Democratic- Republican,George Clinton,NaN,NaN,NaN,NaN,NaN
4,4,5,"March 4, 1817","March 4, 1825",James Monroe,7th United States Secretary of State (1811–...,Democratic- Republican,Daniel D. Tompkins,NaN,NaN,NaN,NaN,NaN


In [74]:
#Pivot Immigration data setdata

immigration = immigration.melt(id_vars=["Year"], 
        var_name="Enforcement Status", 
        value_name="Records")

<ipython-input-74-02f3bdda8864>:3: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resultiing Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  immigration = immigration.melt(id_vars=["Year"],


Since the term dates in the Presidential term dataset only has the starting and ending years, we will need to create records for each year within each term so that the years can be joined to the immigration dataset. We will also need to subtract 1 year from the end of each term to prevent terms from overlapping.

First, we subtract the last year from each term.
Second, we create new records so that we can join this presidential term data to the immigration data by "Year"

In [75]:
#Remove last year from each presidential term to eliminate overlap between terms

terms = terms.fillna(0) #Remova NaN values
terms['start'] =  (pd.to_datetime(terms['start']).dt.year).astype(int) 
terms['end'] = (((pd.to_datetime(terms['end']).dt.year)-1)).astype(int) #Remove one so grouping is clean between presidential terms and annual counts. By subtracting 1 we are discounting the first 3 months leading up to inaguration date of the new president. There are exceptions from this for cases of impeachment and death in office 

#Create new records for years in each term range
terms['Year']=[list(range(x,y+1)) for x , y in zip(terms['start'],terms['end'])]
terms=terms.explode('Year')
#Write to new file
terms.to_csv('terms.csv')

( Exploratory Step )
I found a dataset that gives the total immigrant population in the US by decade from 1920-Present day. I wanted to create a calculation that puts the # of immigration enforcement events into more context.

In [62]:
#Read data
basepop = pd.read_csv('/Users/jackhulbert/Desktop/Data Science Projects/Immigration/Data/Immigration Population.csv')
#Create field for the end of the decade
basepop['end decade'] = basepop['Year']+10

#Create record for each year in each decade so that we can join by Year with our Terms dataset.

basepop['Census Est']=[list(range(x,y+1)) for x , y in zip(basepop['Year'],basepop['end decade'])]
basepop=basepop.explode('Census Est')

#Join data sets on "Year" and convert all values to float.
immigration_merged = immigration.merge(basepop,how='inner',left_on='Year',right_on='Census Est')
immigration_merged['Number of'].apply(lambda x: float(x))

#Compute a "Per Capita" metric for the immigration statistics to give context to our analysis
immigration_merged['Per Capita'] = immigration_merged['Records'] /  (immigration_merged['Number of'])

#Write to new file
immigration_merged.to_csv('pivoted_data.csv')


